# Parte 3: Limpieza del texto para modelado

El preprocesamiento del texto es esencial para reducir el ruido y mejorar la calidad del aprendizaje automático. El objetivo de esta etapa es:

- Eliminar palabras genéricas o irrelevantes
- Reducir vocabulario redundante (palabras con frecuencia 1)
- Filtrar tokens anómalos (muy largos o poco informativos)
- Optimizar el corpus para enfoques como TF-IDF y embeddings

### Importar librerías

In [205]:
!pip install tabulate

Defaulting to user installation because normal site-packages is not writeable


In [206]:
import pandas as pd
import numpy as np
import re
import nltk
from nltk.corpus import wordnet
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from tabulate import tabulate
from collections import Counter

nltk.download('wordnet')
nltk.download('omw-1.4') 
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('averaged_perceptron_tagger_eng')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     C:\Users\alexa\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

### Carga y Limpieza del Dataset

In [207]:
# Cargar el dataset limpio
data_limpia = pd.read_csv('C:\CC219-TP-TF-2024-2--CC92\data\data_limpia.csv')

# Revisar el dataset
print("Dimensiones del dataset:", data_limpia.shape)
print("\nPrimeras filas del dataset:")
data_limpia.head()

<>:2: SyntaxWarning: invalid escape sequence '\C'
<>:2: SyntaxWarning: invalid escape sequence '\C'
C:\Users\alexa\AppData\Local\Temp\ipykernel_23724\1043033834.py:2: SyntaxWarning: invalid escape sequence '\C'
  data_limpia = pd.read_csv('C:\CC219-TP-TF-2024-2--CC92\data\data_limpia.csv')


Dimensiones del dataset: (8469, 11)

Primeras filas del dataset:


,Customer Age,Customer Gender,Product Purchased,Date of Purchase,Ticket Type,Ticket Subject,Ticket Description,Ticket Status,Ticket Priority,Ticket Channel,Hours_to_First_Response
0,32,Other,GoPro Hero,2021-03-22,Technical issue,Product setup,I'm having an issue with the {product_purchase...,Pending Customer Response,Critical,Social media,7.0
1,42,Female,LG Smart TV,2021-05-22,Technical issue,Peripheral compatibility,I'm having an issue with the {product_purchase...,Pending Customer Response,Critical,Chat,6.0
2,48,Other,Dell XPS,2020-07-14,Technical issue,Network problem,I'm facing a problem with my {product_purchase...,Closed,Low,Social media,7.0
3,27,Female,Microsoft Office,2020-11-13,Billing inquiry,Account access,I'm having an issue with the {product_purchase...,Closed,Low,Social media,6.0
4,67,Female,Autodesk AutoCAD,2020-02-04,Billing inquiry,Data loss,I'm having an issue with the {product_purchase...,Closed,Low,Email,20.0


In [208]:
# Definir variables objetivo
# Se consideran "urgentes" los tickets con prioridad "Critical" o "High" (valor 1), el resto son "no urgentes" (valor 0)
data_limpia["Urgency"] = data_limpia["Ticket Priority"].apply(lambda x: 1 if x in ["Critical", "High"] else 0)

# Tickets con tiempo de respuesta igual o menor a 1 hora toman el valor de 1, el resto como 0
data_limpia["Resolution_Time_Bin"] = data_limpia["Hours_to_First_Response"].apply(lambda x: 1 if x <= 1 else 0)

# Filtrar columnas relevantes
data_limpia = data_limpia[["Ticket Description", "Ticket Priority", "Ticket Channel", "Urgency", "Resolution_Time_Bin"]]

In [209]:
# Verificación
total = len(data_limpia)

# Urgencia
urg_counts = data_limpia["Urgency"].value_counts()
print(f"Urgencia:")
print(f"- No urgente (0): {urg_counts[0]} ({urg_counts[0] / total:.2%})")
print(f"- Urgente (1): {urg_counts[1]} ({urg_counts[1] / total:.2%})\n")

# Tiempo de resolución
res_counts = data_limpia["Resolution_Time_Bin"].value_counts()
print(f"Tiempo de resolución:")
print(f"- Más de una hora (0): {res_counts[0]} ({res_counts[0] / total:.2%})")
print(f"- Menos de una hora (1): {res_counts[1]} ({res_counts[1] / total:.2%})")

Urgencia:
- No urgente (0): 4255 (50.24%)
- Urgente (1): 4214 (49.76%)

Tiempo de resolución:
- Más de una hora (0): 8140 (96.12%)
- Menos de una hora (1): 329 (3.88%)


In [210]:
data_limpia.head()

,Ticket Description,Ticket Priority,Ticket Channel,Urgency,Resolution_Time_Bin
0,I'm having an issue with the {product_purchase...,Critical,Social media,1,0
1,I'm having an issue with the {product_purchase...,Critical,Chat,1,0
2,I'm facing a problem with my {product_purchase...,Low,Social media,0,0
3,I'm having an issue with the {product_purchase...,Low,Social media,0,0
4,I'm having an issue with the {product_purchase...,Low,Email,0,0


Se crean dos variables clave para entrenar los modelos de clasificación:
- Urgency: marca los tickets como urgentes (1) si su prioridad es Critical o High, y como no urgentes (0) en otros casos.
- Resolution_Time_Bin: indica si el ticket recibió una primera respuesta en menos de una hora (1) o no (0).

Luego, se filtran las columnas más relevantes: la descripción del ticket, el canal por el que ingresó y las dos nuevas etiquetas objetivo.

Estas variables son relevantes porque se relacionan directamente con los insights obtenidos:

* **Urgency** permite identificar si un ticket es crítico o no según su prioridad. Esto es útil para entrenar modelos que ayuden a mejorar la **priorización automática de tickets urgentes** (Insight 1).

* **Resolution\_Time\_Bin** indica si la primera respuesta fue rápida (menos de una hora). Al relacionarla con el canal de entrada, permite evaluar el **desempeño de los canales** y encontrar oportunidades de mejora (Insight 2).

* **Ticket Description** proporciona el texto del problema, que puede contener patrones comunes relacionados con urgencia o demoras, ayudando a **detectar temas frecuentes y optimizar la respuesta** (Insight 3).


### Preprocesamiento de Datos

In [211]:
stop_words = set(stopwords.words('english'))

# Limpieza de texto
def clean_text(text):
    text = str(text)
    text = re.sub(r'\{.*?\}', '', text)                    # Eliminar placeholders como {product_purchased}
    text = re.sub(r'[^a-zA-Z\s]', '', text)                # Eliminar todo excepto letras y espacios
    text = text.lower()                                    # Pasar todo a minúsculas
    text = re.sub(r'\b\w{1,2}\b', '', text)                # Eliminar palabras de 1 o 2 letras (como "i", "as", "is")
    text = re.sub(r'\s+', ' ', text)                       # Reemplazar múltiples espacios por uno solo
    words = text.split()                                   # Divide el texto en palabras individuales
    words = [w for w in words if w not in stop_words]      # Elimina stopwords
    return ' '.join(words)                                 # Reconstruye el texto limpio como una sola cadena

data_limpia["Ticket Description"] = data_limpia["Ticket Description"].apply(clean_text)

# Codificación de variables categóricas
categorical_features = ["Ticket Priority", "Ticket Channel"]
categorical_transformer = Pipeline(steps=[
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Pipeline para vectorizar el texto con TF-IDF (limita a las 5000 palabras más relevantes y elimina stopwords adicionales)
text_transformer = Pipeline(steps=[
    ('tfidf', TfidfVectorizer(max_features=5000, stop_words='english'))])

# Combinación de transformadores: texto (TF-IDF) + variables categóricas (one-hot)
preprocessor = ColumnTransformer(
    transformers=[
        ('tfidf', text_transformer, "Ticket Description"),
        ('cat', categorical_transformer, categorical_features)])

In [212]:
# Aplicar el preprocesamiento a una muestra
X_preprocessed = preprocessor.fit_transform(data_limpia)

# Imprimir forma del resultado
print("Forma del conjunto transformado:", X_preprocessed.shape)

# Verificar limpieza de texto
print(data_limpia["Ticket Description"].sample(1).values[0])

# Verificar categorías detectadas
ohe = preprocessor.named_transformers_['cat']['onehot']
print("\nCategorías codificadas:", ohe.get_feature_names_out(categorical_features)[:3])

# Verificar palabras clave extraídas del texto
tfidf = preprocessor.named_transformers_['tfidf']['tfidf']
print("Palabras TF-IDF más comunes:", tfidf.get_feature_names_out()[:5])


Forma del conjunto transformado: (8469, 5008)
ive encountered data loss issue files documents seem disappeared guide retrieve meantime ill continue develop new products thank ive checked available software updates none

Categorías codificadas: ['Ticket Priority_Critical' 'Ticket Priority_High' 'Ticket Priority_Low']
Palabras TF-IDF más comunes: ['ability' 'able' 'abovementioned' 'abovethen' 'abroad']


### Interpretación de la salida transformada

- **Forma del conjunto transformado: (8469, 5008)**  
  Esto indica que, después de aplicar la transformación de texto y la codificación de variables categóricas, contamos con un total de 8469 ejemplos (tickets) y 5008 características distintas por ejemplo. Estas características incluyen tanto las palabras relevantes del texto como las variables categóricas transformadas (como prioridad o canal del ticket).

- **Ejemplo de texto limpio:**  
  Se muestra un ejemplo del contenido de un ticket después de la limpieza. En esta etapa, se eliminaron caracteres especiales, palabras vacías (stopwords), y palabras poco informativas, dejando solo los términos relevantes para el modelo. Esto mejora la calidad de la representación del texto.

- **Categorías codificadas:**  
  Se listan las variables categóricas transformadas mediante codificación one-hot. En este caso, corresponden a los diferentes niveles de prioridad del ticket. Cada una se convierte en una columna binaria que indica su presencia.

- **Palabras TF-IDF más comunes:**  
  Aquí se muestran algunas de las palabras con mayor peso o frecuencia relativa dentro del corpus, después de aplicar la vectorización TF-IDF. Estas palabras representan los términos más relevantes para el conjunto de tickets según su frecuencia y distribución.

In [213]:
vectorizer = preprocessor.named_transformers_['tfidf'].named_steps['tfidf']
feature_names = vectorizer.get_feature_names_out()
idf_scores = vectorizer.idf_

# Mostrar top 20 palabras con menor IDF (más frecuentes)
top_indices = np.argsort(idf_scores)[:20]
top_words = feature_names[top_indices]

print("Palabras más comunes según TF-IDF:", top_words)

Palabras más comunes según TF-IDF: ['issue' 'assist' 'ive' 'problem' 'product' 'software' 'steps' 'data'
 'persists' 'noticed' 'resolve' 'update' 'device' 'help' 'unable' 'tried'
 'started' 'need' 'times' 'using']


Imprimir las 20 palabras más comunes nos permite tener una visión general de qué términos están teniendo mayor peso dentro del corpus de texto. Esta revisión es útil para validar si el proceso de limpieza textual fue efectivo, así como para identificar si las palabras que destacan son realmente relevantes para el análisis. También puede ayudarnos a detectar ruido, redundancias o términos poco informativos que sería conveniente filtrar antes de entrenar un modelo.

Estas palabras reflejan un patrón: 
- Los clientes reportan problemas técnicos (bugs, errores en software/dispositivos).
- Muchos buscan ayuda inmediata (“help”, “resolve”, “unable”, “need”).
- Algunos describen acciones previas intentadas (“tried”, “steps”, “started”).

Este hallazgo respalda el Insight 3 del EDA: la ausencia de un problema dominante refleja múltiples frentes de mejora operativa.

La diversidad de palabras recurrentes sugiere que los clientes enfrentan una amplia gama de inconvenientes, no un único problema repetido. Por tanto, una solución automatizada debe aprender a priorizar casos basándose en estas palabras clave y su contexto.

In [214]:
data_limpia["Ticket Description"].sample(5, random_state=1)

1825    issue please assist thanks return thanks under...
473     issue please assist protected stringproperty n...
3509    encountering software bug whenever try perform...
5415    issue please assist please help order fulfillm...
5537    issue please assist gnome mmt addressing servi...
Name: Ticket Description, dtype: object

In [215]:
generic_phrases = ['assist', 'please', 'help', 'issue', 'thanks', 'thank', 'sorry', 'ive' ]
def remove_generic_phrases(text):
    for phrase in generic_phrases:
        text = text.replace(phrase, '')
    return text

data_limpia["Ticket Description"] = data_limpia["Ticket Description"].apply(remove_generic_phrases)
data_limpia["Ticket Description"].sample(5, random_state=1)

1825        return  understanding  sharing   forget   ...
473        protected stringproperty name productpurcha...
3509    encountering software bug whenever try perform...
5415         order fulfillment process  patience patie...
5537       gnome mmt addressing service found device s...
Name: Ticket Description, dtype: object

In [216]:
todas_las_palabras = ' '.join(data_limpia["Ticket Description"]).split()

# Contar frecuencia de cada palabra
word_freq = Counter(todas_las_palabras)

# 1. Palabras con frecuencia 1
palabras_frec_1 = [word for word, freq in word_freq.items() if freq == 1]

# 2. Palabras con más de 25 caracteres
palabras_largas = [word for word in word_freq if len(word) > 25]

# 3. Palabras que contienen algo no alfabético
palabras_no_alfabeticas = [word for word in word_freq if not word.isalpha()]

# Imprimir resumen
print(f"Número de palabras con frecuencia 1: {len(palabras_frec_1)}")
print(f"Número de palabras con más de 25 caracteres: {len(palabras_largas)}")
print(f"Número de palabras con caracteres no alfabéticos: {len(palabras_no_alfabeticas)}")


Número de palabras con frecuencia 1: 3716
Número de palabras con más de 25 caracteres: 161
Número de palabras con caracteres no alfabéticos: 0


Aunque se realizó una limpieza básica del texto (eliminando caracteres no alfabéticos, palabras cortas y stopwords), aún persisten ciertos indicadores de "ruido" residual en el corpus. Por ejemplo, se detectaron 3720 palabras con frecuencia 1, lo que puede incluir errores tipográficos, nombres propios poco comunes o tokens irrelevantes. Además, se identificaron 162 palabras con más de 25 caracteres, que podrían ser strings concatenados, identificadores únicos o errores de tokenización. Para reducir aún más el ruido, también se eliminaron frases genéricas frecuentes como "assist", "please", "help", "thanks", entre otras. 

In [217]:
# Filtrar cada texto eliminando palabras con frecuencia 1 o demasiado largas
def limpiar_palabras_residuales(text):
    return " ".join([
        word for word in text.split()
        if word_freq[word] > 1 and len(word) <= 25
    ])

# Aplicar la limpieza
data_limpia["Ticket Description"] = data_limpia["Ticket Description"].apply(limpiar_palabras_residuales)

all_words_cleaned = " ".join(data_limpia["Ticket Description"]).split()
freq_cleaned = Counter(all_words_cleaned)

# Estadísticas finales
unicas_restantes = len(freq_cleaned)
largas_restantes = sum(1 for word in freq_cleaned if len(word) > 25)

print(f"Número de palabras únicas después de limpiar: {unicas_restantes}")
print(f"Número de palabras con más de 25 caracteres después de limpiar: {largas_restantes}")


Número de palabras únicas después de limpiar: 2801
Número de palabras con más de 25 caracteres después de limpiar: 0


Reducir la cantidad de palabras únicas era necesario para eliminar ruido, mejorar la calidad del análisis y optimizar el rendimiento del modelo. Depende del objetivo y del tamaño del corpus, pero en general sí conviene eliminar palabras con frecuencia 1 si representan una proporción significativa (como en este caso: 3720 de más de 5000). Estas palabras suelen ser errores tipográficos, términos irrelevantes o muy específicos, y en modelos basados en bolsa de palabras o TF-IDF aportan muy poco valor o solo introducen ruido. Al limpiar el vocabulario, se obtiene una representación más consistente y útil del texto, lo que facilita el aprendizaje del modelo y reduce la complejidad computacional.

In [218]:
lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN  

def lemmatize(text):
    text = str(text)
    tokens = word_tokenize(text)
    tagged_tokens = pos_tag(tokens, lang='eng')
    lemmatized = [lemmatizer.lemmatize(word, get_wordnet_pos(pos)) for word, pos in tagged_tokens]
    return ' '.join(lemmatized)

data_limpia["Ticket Description"] = data_limpia["Ticket Description"].apply(lemmatize)

In [242]:
# Definir listas de palabras clave con diferente peso semántico
urgency_keywords_strong = [
    'urgent', 'immediately', 'asap', 'critical', 'emergency',
    'crash', 'system down', 'not working', 'broken', 'malfunction',
    'lose data', 'lost data', 'data loss', 'blocked', 'unresponsive',
    'no solution', 'bug', 'glitch', 'failure', 'can’t use', 'not responding',
    'not function', 'battery drain', 'sudden drop', 'hardware problem'
]

urgency_keywords_weak = [
    'problem', 'issue', 'error', 'didnt work', 'try', 'unable',
    'update', 'reset', 'settings', 'not working properly', 'slow',
    'not stable', 'intermittent', 'trouble', 'step', 'contact support',
    'strange', 'noise', 'performance', 'use', 'device', 'software',
    'notice', 'change', 'since update', 'after update'
]

# Función de detección de urgencia con puntuación ponderada
def detect_urgencia_score(text):
    text = str(text).lower()
    score = 0

    # Palabras fuertes (suman 2 puntos)
    for keyword in urgency_keywords_strong:
        if re.search(r'\b' + re.escape(keyword) + r'\b', text):
            score += 2

    # Palabras débiles (suman 1 punto)
    for keyword in urgency_keywords_weak:
        if re.search(r'\b' + re.escape(keyword) + r'\b', text):
            score += 1

    # Umbral dinámico ajustado desde el texto
    return 1 if score >= 6 else 0  # Ajusta este umbral según tus resultados

# Manejo de valores nulos
data_limpia["Ticket Description"] = data_limpia["Ticket Description"].fillna("sin descripcion")

# Aplicar función al dataset
data_limpia['Urgency_from_Text'] = data_limpia['Ticket Description'].apply(detect_urgencia_score)

In [243]:
# Identificar casos donde Urgency y Urgency_from_Text no coinciden
data_limpia['Coincide'] = (data_limpia['Urgency'] == data_limpia['Urgency_from_Text'])

# Filtrar tickets donde haya desacuerdo
no_coinciden = data_limpia[~data_limpia['Coincide']]

# Contar cuántos casos coinciden entre Urgency y Urgency_from_Text
coincidencias = (data_limpia['Urgency'] == data_limpia['Urgency_from_Text']).sum()
total_tickets = len(data_limpia)

# Mostrar resultado
print(f"Total de tickets: {total_tickets}")
print(f"Tickets donde Urgency y Urgency_from_Text coinciden: {coincidencias}")
print(f"Porcentaje de coincidencia: {(coincidencias / total_tickets) * 100:.2f}%")

# Guardar estos casos para revisión
no_coinciden.to_csv('C:\CC219-TP-TF-2024-2--CC92\data/tickets_no_coinciden.csv', index=False)

Total de tickets: 8469
Tickets donde Urgency y Urgency_from_Text coinciden: 4301
Porcentaje de coincidencia: 50.79%


<>:17: SyntaxWarning: invalid escape sequence '\C'
<>:17: SyntaxWarning: invalid escape sequence '\C'
C:\Users\alexa\AppData\Local\Temp\ipykernel_23724\1814054218.py:17: SyntaxWarning: invalid escape sequence '\C'
  no_coinciden.to_csv('C:\CC219-TP-TF-2024-2--CC92\data/tickets_no_coinciden.csv', index=False)


Tras analizar el archivo, pudimos extraer conclusiones como:
- Muchos tickets marcados como "Critical" o "High" no tienen lenguaje urgente.
- Algunos tickets con frases como “system down”, “lost data”, “not working” están siendo ignorados por tener prioridad baja.

In [244]:
# Cargar datos
data_no_coinciden = pd.read_csv('C:/CC219-TP-TF-2024-2--CC92/data/tickets_no_coinciden.csv')

# Separar los tickets según tipo de discrepancia
caso_A = data_no_coinciden[data_no_coinciden['Urgency'] == 1]   # Urgente manual, no detectado en texto
caso_B = data_no_coinciden[data_no_coinciden['Urgency_from_Text'] == 1]  # Urgente en texto, no marcado manualmente

# Seleccionar solo las columnas relevantes para mostrar
columns_to_show = ["Ticket Description", "Ticket Priority", "Ticket Channel", "Urgency", "Urgency_from_Text"]

print("\n📌 Caso A: Tickets marcados como urgentes (manual) pero NO identificados como tales desde el texto")
print(tabulate(caso_A[columns_to_show].head(5), headers="keys", tablefmt="psql", showindex=False))

print("\n📌 Caso B: Tickets con urgencia detectada en texto pero NO priorizados manualmente")
print(tabulate(caso_B[columns_to_show].head(5), headers="keys", tablefmt="psql", showindex=False))

# Extraer palabras de tickets urgentes no identificados manualmente
urgentes_no_detectados = caso_B['Ticket Description'].str.lower().str.cat(sep=' ')
palabras = re.findall(r'\b\w{3,}\b', urgentes_no_detectados)

# Contar palabras más frecuentes
word_counts = Counter(palabras).most_common(20)
print("\nPalabras más comunes en tickets urgentes no detectados:")
print(word_counts)


📌 Caso A: Tickets marcados como urgentes (manual) pero NO identificados como tales desde el texto
+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+------------------+-----------+---------------------+
| Ticket Description                                                                                                                                                                             | Ticket Priority   | Ticket Channel   |   Urgency |   Urgency_from_Text |
|------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-------------------+------------------+-----------+---------------------|
| bill code appreciate request website address double check email address try troubleshooting ste

**Caso A: Urgencia marcada manualmente pero NO detectada desde el texto**

Estos tickets fueron clasificados como urgentes por criterio humano (prioridad "Critical" o "High"), pero no contienen señales claras de urgencia en su descripción textual .
Es decir:
- El cliente no usó términos como "urgent", "immediately", "system down", etc.
- La urgencia fue determinada sin un análisis claro del problema.
- Esto refuerza el Insight 1 : “La gestión manual de prioridades no garantiza atención prioritaria a casos realmente críticos.”

**Caso B: Urgencia detectada en el texto pero NO priorizada manualmente**

Estos tickets contienen lenguaje claramente urgente ("problem", "doesnt respond", "factory reset", "remains unresolved") pero fueron etiquetados como no urgentes (Low o Medium) por los operadores humanos .
Esto confirma el Insight 1 y respalda aún más la necesidad de un sistema automatizado:
- Los agentes no siempre logran identificar la gravedad real de los problemas.
- Algunos clientes expresan con claridad situaciones críticas que afectan su experiencia, pero no reciben respuesta inmediata.

In [245]:
# Crear la columna Urgency_Final basada en un OR lógico entre ambas fuentes
data_limpia['Urgency_Final'] = data_limpia.apply(
    lambda row: 1 if (row['Urgency'] == 1 or row['Urgency_from_Text'] == 1) else 0,
    axis=1
)

La columna Urgency_Final representa una clasificación combinada de urgencia.
Se marca como urgente (1) si al menos una de las dos fuentes lo considera urgente:

- Urgency: etiqueta apartir de la clasificación del dataset, se colocó "Urgente" (1) a los que estan clasificados como "Critial" o "Hight" y "No Urgente" (0), "Low" o "Medium". 
- Urgency_from_Text: etiqueta automática basada en el contenido del texto del ticket.

Si ambas lo consideran no urgente (0), entonces se marca como no urgente (0).

Este enfoque busca no pasar por alto ningún posible caso urgente, combinando la evaluación humana con la automatizada mediante una lógica de tipo "OR" lógico.

In [247]:
data_limpia.to_csv('C:\CC219-TP-TF-2024-2--CC92\data/data_limpia2.csv', index=False)

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\alexa\AppData\Local\Temp\ipykernel_23724\1558628782.py:1: SyntaxWarning: invalid escape sequence '\C'
  data_limpia.to_csv('C:\CC219-TP-TF-2024-2--CC92\data/data_limpia2.csv', index=False)
